# Birdsongs - 09.- Modelando - Determinando Sets Train, Validation y Test


### Determinar los sets de datos

A partir de los datos procesados, crea los directorios de Train, Validación y Test. Crea una copia de los datos sin modificar los datos originales. Cada vez que se utilice este notebook generará un juego de datos nuevo o sustituirá a uno existente. Hay que tenerlo en cuenta si no disponemos suficiente espacio para su tratamiento.

![setmodel](./resources/train_test_validation.png)


* https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets
* http://tarangshah.com/blog/2017-12-03/train-validation-and-test-sets/
* http://mateos.io/blog/train-test-split/



### Determinar el tamaño de cada set de datos

Se establecen los ratios del tamaño de cada dataset en función de los criterios normales que utilizan otras personas en sus estudios. Si alguno de los ratios se establece a 0, significa que no queremos datos para este set y obviamos su tratamiento.

* https://www.beyondthelines.net/machine-learning/how-to-split-a-dataset/
* https://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validatio
* https://www.researchgate.net/post/Is_there_an_ideal_ratio_between_a_training_set_and_validation_set_Which_trade-off_would_you_suggest
* https://www.researchgate.net/post/can_someone_recommend_what_is_the_best_percent_of_divided_the_training_data_and_testing_data_in_neural_network_7525_or_8020_or_9010


>There are two competing concerns: with less training data, your parameter estimates have greater variance. With less testing data, your performance statistic will have greater variance. Broadly speaking you should be concerned with dividing data such that neither variance is too high, which is more to do with the absolute number of instances in each category rather than the percentage.
>
>If you have a total of 100 instances, you're probably stuck with cross validation as no single split is going to give you satisfactory variance in your estimates. If you have 100,000 instances, it doesn't really matter whether you choose an 80:20 split or a 90:10 split (indeed you may choose to use less training data if your method is particularly computationally intensive).
>
>Assuming you have enough data to do proper held-out test data (rather than cross-validation), the following is an instructive way to get a handle on variances:
>
>* Split your data into training and testing (80/20 is indeed a good starting point)
>* Split the training data into training and validation (again, 80/20 is a fair split).
>* Subsample random selections of your training data, train the classifier with this, and record the performance on the validation set
>* Try a series of runs with different amounts of training data: randomly sample 20% of it, say, 10 times and observe performance on the validation data, then do the same with 40%, 60%, 80%. You should see both greater performance with more data, but also lower variance across the different random samples
>* To get a handle on variance due to the size of test data, perform the same procedure in reverse. Train on all of your training data, then randomly sample a percentage of your validation data a number of times, and observe performance. You should now find that the mean performance on small samples of your validation data is roughly the same as the performance on all the validation data, but the variance is much higher with smaller numbers of test samples""


Para generar los diferentes datasets, utilizaremos los ID de las grabaciones; esto significa que si hemos dividido una grabación en ficheros de menor tamaño, todos los ficheros que tengamos de la grabación irán al mismo set al que haya sido asignado. Así evitaremos redundancia de datos.

### Resultado

El resultado de este notebook es la generación de una estructura de directorios del tipo:

    ./modelrootpath                     
        /modelpath
            /train
            /test
            /validation




## 1.-  Librerías

Carga las librerías que se van a usar en el notebook

Para dividir el dataset en los sets de train, test y validación utilizamos
[**scikit-learn**](https://scikit-learn.org/stable) 

> A set of python modules for machine learning and data mining

      conda install scikit-learn 




In [15]:
# importar librerías
import os
import shutil
import glob

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


## 2.-  Carga dataset

Carga el dataset en un dataframe de pandas

In [16]:
# carga dataset
csv_path = 'Birdsongs_My_Birdsongs_Europe_20181230103204.csv'
df_birdsongs = pd.read_csv(csv_path)


## 3.-  Filtra por tipo de canto (si procede)

Crea un dataframe filtrando por un tipo de canto en concreto, procedemos a seleccionar sólo aquellas grabaciones que pertenecen al tipo de canto seleccionado.  Tipos de cantos normalizados:

* call.- llamada
* song.- canto
* icall.- incluye llamada
* isong.- incluye canto
* other.- otros tipos 




In [17]:
# filtra dataframe filtrando por tipo de canto
filter_type = ''
#filter_type = 'song'

if filter_type.strip() != '':
    df_birdsongs = df_birdsongs.loc[df_birdsongs['nType'] == filter_type]


## 4.- Normaliza dataset

Si se realizan filtrados sobre el dataset es posible que existan especies que pierdan significancia o que dejen de tener datos. Para evitar problemas con estas especies, se eliminarán del dataset si el número de muestras es inferior a 10. 

In [18]:
# dataframe de especies
df_species = df_birdsongs.groupby('Name')['ID'].count()
df_species = df_species.reset_index()
df_species.columns = ['Name', 'Count'] 

print("Before filtering")
print(df_species.describe())

# selecciona especies con más de 10 grabaciones
min_records = 10
df_species = df_species.loc[df_species['Count'] > min_records]

print("\nAfter filtering")
print(df_species.describe())

# elimina aquellas grabaciones de las especies que hemos eliminado del dataset
df_birdsongs.drop(df_birdsongs.loc[~df_birdsongs['Name'].isin(list(df_species['Name'].values))].index, inplace=True)


Before filtering
            Count
count  103.000000
mean   168.640777
std     52.092086
min    101.000000
25%    122.000000
50%    155.000000
75%    214.000000
max    250.000000

After filtering
            Count
count  103.000000
mean   168.640777
std     52.092086
min    101.000000
25%    122.000000
50%    155.000000
75%    214.000000
max    250.000000


## 5.-  Filtra por especies (permite seleccionar muestras).  

Permite generar un juego de datos para unas especies concretas o para todo el dataset. Es posible que para realizar pruebas o por el volumen de datos, deseemos utilizar un número menor de especies, para entrenar el modelo. Esto será posible cambiando el parámetro de configuración.

Se muestra posteriormente el porcentaje correspondiente a cada especie, para ver cómo de balanceadas están las clases.

In [19]:
# número de especies (si son todas, contar los registros del dataset)
sample = df_species['Name'].count()
random_state = 333

# crea dataframe seleccionando el número de especies determinado
df_sample = df_species.sample(n=sample, random_state=random_state)

# calcula el porcentaje que representa cada especie sobre el tamaño del dataset
df_sample['Porcentage'] = (df_sample['Count'] / df_sample['Count'].sum()) * 100
print(df_sample.describe())

# ordena el dataframe en orden descendente
df_sample.sort_values(by='Porcentage', ascending=False, inplace=True)

print(df_sample)


            Count  Porcentage
count  103.000000  103.000000
mean   168.640777    0.970874
std     52.092086    0.299897
min    101.000000    0.581462
25%    122.000000    0.702360
50%    155.000000    0.892343
75%    214.000000    1.232009
max    250.000000    1.439263
                           Name  Count  Porcentage
86               Sitta europaea    250    1.439263
18                 Cettia cetti    250    1.439263
37          Emberiza citrinella    250    1.439263
97      Troglodytes troglodytes    250    1.439263
75       Phylloscopus trochilus    250    1.439263
29          Cyanistes caeruleus    250    1.439263
43            Fringilla coelebs    250    1.439263
19              Chloris chloris    250    1.439263
71       Phylloscopus collybita    250    1.439263
64                  Parus major    250    1.439263
55            Loxia curvirostra    250    1.439263
40           Erithacus rubecula    250    1.439263
67               Periparus ater    250    1.439263
91           Syl

In [20]:
# filtra el dataset con las especies seleccionadas
df_birdsongs.drop(df_birdsongs.loc[~df_birdsongs['Name'].isin(list(df_sample['Name'].values))].index, inplace=True)


## 6.- Genera los sets de Train, Validation y Test

Para generar los distintos sets al azar vamos a utilizar un módulo de sklearn que nos facilita el trabajo

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

>Split arrays or matrices into random train and test subsets

Crea una matriz de "features" con los ID de las grabaciones y un vector de "labels" con los nombres de las especies a las que pertenecen y a partir de ellas se generan los distintos sets

In [21]:
# Crea matriz de features y vector de labels
X = df_birdsongs['ID'].values
y = df_birdsongs['Name'].values


Utiliza **train_test_split** para generar los sets. Se ejecuta en dos pasos:
* Genera el set de train y test
* Divide el set de train en train y validation

El resultado son tres listas, cada una de ellas con los ID de las grabaciones


In [22]:
# establece ratios de cada set
train_rate = 0.65
validation_rate = 0.15
test_rate = 0.20

# crea set de train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_rate, random_state=1, stratify=y)

# divide set de train en train y validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_rate, random_state=1, stratify=y_train)

# grabaciones en el set de train
l_train = list(X_train)

# grabaciones en el set de validación
l_val = list(X_val)

# grabaciones en el set de test
l_test = list(X_test)


## 7.- Actualiza dataset con el set asignado a cada grabación

Crea una nueva columna en el dataset indicando a que set pertenece la grabación

In [23]:
#----------------------------------------------------------------------------
# set_trainvalidationtest(ID)
#  argumentos: 
#      ID: identificador de la grabación
#  retorna: 
#      Set en el que está localizada
#----------------------------------------------------------------------------
def set_trainvalidationtest(ID):
    if ID in l_train:
        return "train"
    
    if ID in l_val:
        return "validation"
    
    if ID in l_test:
        return "test"


In [25]:
# crea columna con el set
df_birdsongs['set'] = df_birdsongs['ID'].apply(set_trainvalidationtest)


## 8.-  Crea directorios de Train, Test y Validación

Crea la estructura de directorios donde vamos a copiar las grabaciones para cada uno de los sets. La estructura es del tipo 

    ./modelrootpath                     
        /modelpath
            /train
            /test
            /validation


**Atención. Si el directorio /modelpath ya existe, lo primero que hace es borrarlo, ya que se entiende que se quiere regenerar de nuevo el directorio**


In [28]:
# directorio raíz donde se alojan los datos de los modelos
modelrootpath = './model'

# directorio a crear con el juego de datos
modelpath = os.path.join(modelrootpath, 'melsxxx')

# crea el directorio raíz si no existe
if not os.path.exists(modelrootpath):
    os.mkdir(modelrootpath)

# borra el directorio del modelo si existe y lo vuelve a generar
if os.path.exists(modelpath):
    shutil.rmtree(modelpath)
    
os.mkdir(modelpath)

# crea directorios de cada set
# train 
if train_rate > 0:
    traindir = os.path.join(modelpath, "train")
    if not os.path.exists(traindir):
        os.mkdir(traindir)

# validation 
if validation_rate > 0:
    valdir = os.path.join(modelpath, "validation")
    if not os.path.exists(valdir):
        os.mkdir(valdir)

# test 
if test_rate > 0:
    testdir = os.path.join(modelpath, "test")
    if not os.path.exists(testdir):
        os.mkdir(testdir)       
        

## 9.- Copia grabaciones a cada set correspondiente

El proceso itera sobre el dataset y por cada una de las grabaciones, copia del directorio origen los datos correspondiente a la grabación en el directorio destino donde haya sido asignado (train, test, validation).






In [12]:
# repositorio con los datos a copiar
datapath= './image/mels'

# itera sobre el dataset y va copiando los ficheros de datos (train, test, validation)
for index, row in df_birdsongs.iterrows():
    # grabación
    ID = row['ID']
    specie = row['Name']
    set_folder = row['set']

    # selecciona todos los ficheros pertenecientes a la grabación
    datadir = os.path.join(datapath, specie)
    files = os.path.join(datadir, ID + '*')

    # crea directorio si no existe
    dest_dir = os.path.join(modelpath, set_folder, specie)
    if not os.path.exists(dest_dir):
        os.mkdir(dest_dir)  
    
    # copia ficheros
    for file in glob.glob(files):
        print(file, 'to', dest_dir, end='\r', flush=True)
        shutil.copy(file, dest_dir)    
    